### 01 packages

In [38]:
################################################################################################
################################################################################################

import numpy as np

################################################################################################
################################################################################################

import pyarrow.parquet as pq

import pandas as pd

################################################################################################
################################################################################################
 
import folium

################################################################################################
################################################################################################

import copy

################################################################################################
################################################################################################
 
import geopandas as gp

from shapely.geometry import Polygon, Point

################################################################################################
################################################################################################

import pulp

################################################################################################
################################################################################################

import networkx as nx

################################################################################################
################################################################################################

import matplotlib

import matplotlib.pyplot as plt

from matplotlib.ticker import MultipleLocator, FormatStrFormatter

import seaborn as sns

from matplotlib.transforms import Bbox

################################################################################################
################################################################################################

from model.Matching import Bipartite_matching

################################################################################################
################################################################################################

import time

################################################################################################
################################################################################################

import scipy.stats as stats

################################################################################################
################################################################################################


import warnings

warnings.filterwarnings('ignore')

### 02 parameter

In [43]:
################################################################################################
################################################################################################

day=1

################################################################################################
################################################################################################

alpha_mu_0,alpha_sigma_0 = 0.0,0.5

alpha_mu_1,alpha_sigma_1 = 3.2,0.2

alpha_mu_2,alpha_sigma_2 = 0.6,0.1

u_decline_mu,u_decline_sigma = -13.2,2.0


### 03 access the manhattan network

In [31]:
################################################################################################
################################################################################################

taxi_zone = gp.read_file('./01data/00taxi_zones/taxi_zones.shp')

taxi_zone = taxi_zone.to_crs('EPSG:4326')

taxi_zone=taxi_zone.loc[taxi_zone.borough=='Manhattan']

islands=[104,103,105,202,194,153]

taxi_zone=taxi_zone.loc[~taxi_zone['LocationID'].isin(islands)]

taxi_zone=taxi_zone[['borough','zone','geometry','LocationID']]

taxi_zone=taxi_zone.reset_index(drop=True)

taxi_zone=taxi_zone[['LocationID','zone','geometry']]

################################################################################################
################################################################################################

def Get_POLYGON(coords):
    
    if coords.type=='Polygon':
        
        return coords
    
    else:
        
        Score={i:coords.geoms[i].area for i in range(len(coords.geoms))}
        
        idx=max(Score, key=Score.get)
        
        return coords.geoms[idx]
    
taxi_zone['geometry']=taxi_zone.apply(lambda x:Get_POLYGON(x['geometry']),axis=1)

################################################################################################
################################################################################################

zone_geometry={}

for idx,row in taxi_zone.iterrows():
    
    zone_geometry[row.LocationID]=row.geometry


################################################################################################
################################################################################################

background = folium.Map([40.769602, -73.973667],tiles='openstreetmap',zoom_start=12)

folium.Choropleth(
    geo_data=taxi_zone[['geometry']],
    fill_color='blue',
    fill_opacity=0.3,
    name='Zone').add_to(background)

folium.LayerControl().add_to(background)

display(background)


### 04 extract the demand from the specific day

In [33]:
################################################################################################
################################################################################################

demand = pq.read_table('./01data/01demand/yellow_tripdata_2023-02.parquet')

################################################################################################
################################################################################################

demand = demand.to_pandas()

################################################################################################
################################################################################################

demand=demand[['tpep_pickup_datetime','PULocationID','DOLocationID']]

################################################################################################
################################################################################################

feasible_zones=taxi_zone.LocationID.to_list()

demand=demand.loc[(demand.PULocationID.isin(feasible_zones))&(demand.DOLocationID.isin(feasible_zones))]

demand['year']=demand.apply(lambda x:x['tpep_pickup_datetime'].year,axis=1)

demand['month']=demand.apply(lambda x:x['tpep_pickup_datetime'].month,axis=1)

demand['day']=demand.apply(lambda x:x['tpep_pickup_datetime'].day,axis=1)

demand['hour']=demand.apply(lambda x:x['tpep_pickup_datetime'].hour,axis=1)

demand['minute']=demand.apply(lambda x:x['tpep_pickup_datetime'].minute,axis=1)

demand['stamp']=demand.apply(lambda x:int(x['tpep_pickup_datetime'].timestamp()),axis=1)

################################################################################################
################################################################################################

demand=demand.loc[(demand.year==2023)&(demand.month==2)&(demand.day==day)]

demand=demand.reset_index(drop=True)

################################################################################################
################################################################################################

demand['request_second']=demand.apply(lambda x:int(x.stamp-x.tpep_pickup_datetime.replace(hour=0, minute=0, second=0, microsecond=0).timestamp()),axis=1)

################################################################################################
################################################################################################

demand

,tpep_pickup_datetime,PULocationID,DOLocationID,year,month,day,hour,minute,stamp,request_second
0,2023-02-01 00:32:53,142,163,2023,2,1,0,32,1675211573,1973
1,2023-02-01 00:52:40,148,236,2023,2,1,0,52,1675212760,3160
2,2023-02-01 00:12:39,137,244,2023,2,1,0,12,1675210359,759
3,2023-02-01 00:56:53,263,141,2023,2,1,0,56,1675213013,3413
4,2023-02-01 00:20:40,48,243,2023,2,1,0,20,1675210840,1240
...,...,...,...,...,...,...,...,...,...,...
92180,2023-02-01 23:14:00,114,238,2023,2,1,23,14,1675293240,83640
92181,2023-02-01 23:46:00,164,42,2023,2,1,23,46,1675295160,85560
92182,2023-02-01 23:15:00,107,50,2023,2,1,23,15,1675293300,83700
92183,2023-02-01 23:25:00,164,79,2023,2,1,23,25,1675293900,84300


### 05 generate the demand as required

\begin{equation}
u_{o,d,i} = \overbrace{\alpha_{0,o}}^{\textrm{constant term}} - \overbrace{\alpha_{1,o} \cdot f_{o,d,i}}^{\textrm{order fare}} - \overbrace{ \alpha_{2,o} \cdot \tau_{o,d} }^{\textrm{pick-up time}}, \forall o \in O,
\label{eq:order utility}
\end{equation}

\begin{equation}
p_{o,d,i} = \frac{\textrm{exp}(u_{o,d,i})}{\textrm{exp}(u_{o}^{\textrm{c}})+\textrm{exp}(u_{o,d,i})+\sum_{d^{'} \in D} [\textrm{exp}(u_{o,d^{'},j}) x_{o,d^{'},j}]},
\label{eq:order prob}
\end{equation}

In [44]:
################################################################################################
################################################################################################

# 01 sample the pickup and dropoff points from the network

def sample_point(polygon):
    
    while True:
        
        min_x, min_y, max_x, max_y = polygon.bounds
        
        random_point = (np.random.uniform(min_x, max_x), np.random.uniform(min_y, max_y))
        
        if polygon.contains(Point(random_point)):
            
            break
            
    return random_point

demand['pickup_point']=demand.apply(lambda x:sample_point(zone_geometry[x.PULocationID]),axis=1)

demand['dropoff_point']=demand.apply(lambda x:sample_point(zone_geometry[x.DOLocationID]),axis=1)

demand['pickup_lng']=demand.apply(lambda x:x['pickup_point'][0],axis=1)

demand['pickup_lat']=demand.apply(lambda x:x['pickup_point'][1],axis=1)

demand['dropoff_lng']=demand.apply(lambda x:x['dropoff_point'][0],axis=1)

demand['dropoff_lat']=demand.apply(lambda x:x['dropoff_point'][1],axis=1)

################################################################################################
################################################################################################

# 02 shortest travel distance

def Get_distance(point1,point2):
    
    return Point(point1).distance(Point(point2))*111000*1.2

demand['distance']=demand.apply(lambda x:Get_distance(x['pickup_point'],x['dropoff_point']),axis=1)

################################################################################################
################################################################################################

# 03 choice preference

def Truncated_Gauss(mu,sigma,scale):
    
    lower,upper=mu-sigma,mu+sigma

    X = stats.truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma)
    
    return X.rvs(scale)

demand['alpha_0']=Truncated_Gauss(alpha_mu_0,alpha_sigma_0,demand.shape[0])

demand['alpha_1']=Truncated_Gauss(alpha_mu_1,alpha_sigma_1,demand.shape[0])

demand['alpha_2']=Truncated_Gauss(alpha_mu_2,alpha_sigma_2,demand.shape[0])

demand['u_decline']=Truncated_Gauss(u_decline_mu,u_decline_sigma,demand.shape[0])

demand['u_decline']=demand.apply(lambda x:x.u_decline*x.distance/1000.0,axis=1)

################################################################################################
################################################################################################

# 04 order ID

demand['order_id']=demand.index

demand['order_id']=demand.apply(lambda x:'o'+str(x.order_id),axis=1)

################################################################################################
################################################################################################


demand=demand[['order_id','request_second','hour','minute',\
               'pickup_lat','pickup_lng','dropoff_lat','dropoff_lng','distance',\
               'alpha_0','alpha_1','alpha_2','u_decline']]

################################################################################################
################################################################################################

demand.to_csv('./01data/01demand/customer_2023-02-'+str(day)+'.csv')

demand

,order_id,request_second,hour,minute,pickup_lat,pickup_lng,dropoff_lat,dropoff_lng,distance,alpha_0,alpha_1,alpha_2,u_decline
0,o0,1973,0,32,40.774166,-73.978016,40.762506,-73.974714,1614.166901,0.433891,3.168427,0.597278,-20.686278
1,o1,3160,0,52,40.717088,-73.992605,40.778119,-73.959438,9252.200584,0.141097,3.226902,0.592721,-133.562377
2,o2,759,0,12,40.736679,-73.973677,40.834976,-73.938357,13912.712850,-0.056330,3.175902,0.574131,-160.116925
3,o3,3413,0,56,40.784183,-73.948801,40.767779,-73.961390,2754.271913,-0.305111,3.355605,0.573620,-34.234043
4,o4,1240,0,20,40.761817,-73.993415,40.855980,-73.939652,14442.988210,0.435935,3.066248,0.619007,-195.141005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92180,o92180,83640,23,14,40.728620,-74.001427,40.790819,-73.975322,8985.054802,-0.363875,3.136359,0.591331,-122.685779
92181,o92181,85560,23,46,40.746439,-73.982684,40.827816,-73.934837,12574.178630,-0.245076,3.199573,0.521853,-181.543077
92182,o92182,83700,23,15,40.736968,-73.978972,40.770094,-73.995771,4947.227765,0.445747,3.345793,0.640830,-62.212503
92183,o92183,84300,23,25,40.747151,-73.983983,40.734003,-73.988926,1870.981674,-0.140684,3.153248,0.591141,-26.596691
